In [1]:
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from pathlib import Path

if sys.version_info[0] < 3:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve


In [2]:
spark = SparkSession.builder \
    .appName("ast-ner")\
    .master("local[1]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultS1ize", "2G") \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:1.6.2")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [3]:
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

In [4]:
import time


embeddingsFile = './PubMed-shuffle-win-2.bin'
embeddingsUrl = 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/PubMed-shuffle-win-2.bin'
# this may take a couple minutes
if not Path(embeddingsFile).is_file():
    urlretrieve(embeddingsUrl, embeddingsFile)

documentAssembler = DocumentAssembler()\
    .setInputCol("sentence")\
    .setOutputCol("document")\
    
tokenizer = Tokenizer()\
    .setInputCols(["document"])\
    .setOutputCol("token")
    
pretrained_pos = PerceptronModel().pretrained()\
    .setInputCols(["document", "token"])\
    .setOutputCol("pos")
    
pretrained_ner = NerCrfModel().pretrained()\
    .setInputCols(["document", "token", "pos"])\
    .setOutputCol("ner")

ner_converter = NerConverter()\
    .setInputCols(["document", "token", "ner"])\
    .setOutputCol("nerconverter")

assertion = AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols(["document", "nerconverter"])\
    .setOutputCol("assertion")\
    .setBatchSize(16)\
    .setEpochs(2)\
    .setEmbeddingsSource(embeddingsFile,200,3)

finisher = Finisher() \
    .setInputCols(["assertion", "ner"]) \
    .setIncludeKeys(False)

pipeline = Pipeline(
    stages = [
    documentAssembler,
    tokenizer,
    pretrained_pos,
    pretrained_ner,
    ner_converter,
    assertion,
    finisher
  ])


In [5]:
#Load the input data to be annotated
data = spark.createDataFrame([
    ["hello Peter how are you? And how is Robert Douglas?", "Affirmed"],
    ["master of yards", "Negated"],
    ["what is this", "Negated"],
    ["Michael Jordan is a good person", "Negated"],
    ["Robert Deniro is not a good person", "Negated"],
    ["Learn from Carlos Rodriguez", "Negated"],
    ["Who is Jhon here?", "Negated"],
    ["my friend Lucas", "Affirmed"]
    ]).toDF("sentence", "label")
data.show()

+--------------------+--------+
|            sentence|   label|
+--------------------+--------+
|hello Peter how a...|Affirmed|
|     master of yards| Negated|
|        what is this| Negated|
|Michael Jordan is...| Negated|
|Robert Deniro is ...| Negated|
|Learn from Carlos...| Negated|
|   Who is Jhon here?| Negated|
|     my friend Lucas|Affirmed|
+--------------------+--------+



In [6]:
start = time.time()
print("Start fitting")
model = pipeline.fit(data)
print("Fitting is ended")
print (time.time() - start)

Start fitting
Fitting is ended
202.4842014312744


In [7]:
model.stages[-1].setIncludeKeys(False)

Finisher_4f649e30392d12467f54

In [8]:
result = model.transform(data)
result.select("sentence", "finished_ner", "finished_assertion").show()

+--------------------+--------------------+------------------+
|            sentence|        finished_ner|finished_assertion|
+--------------------+--------------------+------------------+
|hello Peter how a...|O@I-PER@O@O@O@O@O...|   Negated@Negated|
|     master of yards|               O@O@O|                  |
|        what is this|               O@O@O|                  |
|Michael Jordan is...| I-PER@I-PER@O@O@O@O|           Negated|
|Robert Deniro is ...|I-PER@I-PER@O@O@O...|           Negated|
|Learn from Carlos...|     O@O@I-PER@I-PER|           Negated|
|   Who is Jhon here?|   I-PER@O@I-PER@O@O|   Negated@Negated|
|     my friend Lucas|           O@O@I-PER|           Negated|
+--------------------+--------------------+------------------+

